In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from nba_api.stats.endpoints import leaguedashteamstats
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.endpoints import playbyplayv2 
from nba_api.stats.endpoints import playergamelog

import time

In [ ]:

def get_top_scorers(n=10,season='2024-25'):
    time.sleep(1)

    player_stats = leaguedashplayerstats.LeagueDashPlayerStats(season='2024-25').get_data_frames()[0]
    top_scorers = player_stats.sort_values(by='PTS', ascending=False).head(n)

    return (top_scorers[['PLAYER_NAME', 'PLAYER_ID', 'PTS', 'GP', 'TEAM_ABBREVIATION']].reset_index(drop=True))

def get_sample_games(player_id, n=15):
    time.sleep(1)

    try:
        #get the game log of the given player to check the playbyplay
        game_log = playergamelog.PlayerGameLog(player_id=player_id, season='2024-25').get_data_frames()[0]

        #Get sample games
        sample_games = game_log.head(n)
        return sample_games['Games_ID'].tolist()
    
    except Exception as e: 
        print(f"Error getting games for player {player_id}: {e}")
        return []

In [67]:
def is_missed_shot(play):
    return play.get('EVENTMSGTYPE', 0) == 2

def is_offensive_rebound(play, shooting_team_id):
    event_type = play.get('EVENTMSGTYPE', 0)
    team_id = play.get('PLAYER1_TEAM_ID', 0)
    return event_type == 4 and team_id == shooting_team_id

def is_score(play):
    event_type = play.get('EVENTMSGTYPE', 0)
    return event_type in [1, 3]  # Made field goal or free throw

def extract_points_from_play(play):
    """Extract points scored from a play"""
    description = str(play.get('HOMEDESCRIPTION', '')) + ' ' + str(play.get('VISITORDESCRIPTION', ''))
    
    if '3PT' in description or 'Three Point' in description:
        return 3
    elif 'Free Throw' in description or 'Technical FREQ' in description:
        return 1
    elif any(keyword in description for keyword in ['Field Goal', 'Dunk', 'Layup']):
        return 2
    else:
        return 2  # Default for made shots



In [68]:
"""Analyze a single game for a specific player's Kobe assists"""

def analyze_game_for_player_kobe_assists(game_id, player_name):
    
    time.sleep(1)
    
    try:
        # Get play-by-play data
        pbp_data = playbyplayv2.PlayByPlayV2(game_id=game_id).get_data_frames()[0]
        plays = pbp_data.to_dict('records')
        
        total_kobe_assist_points = 0
        
        for i, play in enumerate(plays):
            # Check if this is our player's missed shot
            if (is_missed_shot(play) and 
                play.get('PLAYER1_NAME', '') == player_name):
                
                shooting_team_id = play.get('PLAYER1_TEAM_ID', 0)
                
                # Look for Kobe assist sequence
                points = check_kobe_assist_sequence(plays, i, shooting_team_id)
                total_kobe_assist_points += points
        
        return total_kobe_assist_points
    
    except Exception as e:
        print(f"Error analyzing game {game_id}: {e}")
        return 0

"""Check if a missed shot leads to a Kobe assist"""
def check_kobe_assist_sequence(plays, missed_shot_index, shooting_team_id):

    total_points = 0
    found_offensive_rebound = False
    
    # Look at next several plays
    for i in range(missed_shot_index + 1, min(len(plays), missed_shot_index + 8)):
        current_play = plays[i]
        
        # Check for offensive rebound
        if is_offensive_rebound(current_play, shooting_team_id):
            found_offensive_rebound = True
            continue
        
        # If we found offensive rebound, look for scores by same team
        if found_offensive_rebound:
            if is_score(current_play):
                scoring_team_id = current_play.get('PLAYER1_TEAM_ID', 0)
                if scoring_team_id == shooting_team_id:
                    points = extract_points_from_play(current_play)
                    total_points += points
                    continue
            
            # Check for possession-ending events
            event_type = current_play.get('EVENTMSGTYPE', 0)
            if event_type == 5:  # Turnover ends possession
                break
            elif event_type == 4:  # Defensive rebound ends our possession
                rebounding_team = current_play.get('PLAYER1_TEAM_ID', 0)
                if rebounding_team != shooting_team_id:
                    break
        
        # If no offensive rebound found yet and we hit a clear possession change
        if not found_offensive_rebound:
            event_type = current_play.get('EVENTMSGTYPE', 0)
            if event_type in [4, 5]:  # Rebound or turnover
                break
    
    return total_points if found_offensive_rebound else 0

In [69]:
"""Calculate average Kobe assists per game for a player"""
def calculate_player_kobe_assist_average(player_name, player_id, sample_size=15):

    print(f"Calculating Kobe assists for {player_name}...")
    
    # Get sample games
    game_ids = get_sample_games(player_id, sample_size)
    
    if not game_ids:
        return 0.0
    
    total_points = 0
    successful_games = 0
    
    for game_id in game_ids:
        points = analyze_game_for_player_kobe_assists(game_id, player_name)
        total_points += points
        successful_games += 1
        
        # Progress indicator
        print(f"  Game {successful_games}/{len(game_ids)}: {points} Kobe assist points")
    
    average = total_points / successful_games if successful_games > 0 else 0.0
    print(f"  Average: {average:.2f} Kobe assist points per game")
    
    return average

"""Calculate Kobe assist averages for top scorers"""
def calculate_top_scorers_kobe_assists(n_players=10, sample_size=10):

    print("=" * 50)
    print("NBA Kobe Assist Analysis - Top Scorers")
    print("=" * 50)
    
    # Get top scorers
    top_scorers = get_top_scorers(n_players)
    print(f"Analyzing top {n_players} scorers with {sample_size} games each...")
    print()
    
    results = {}
    
    for idx, player in top_scorers.iterrows():
        player_name = player['PLAYER_NAME']
        player_id = player['PLAYER_ID']
        ppg = player['PTS']/player['GP']
        
        print(f"{idx + 1}. {player_name} ({ppg:.1f} PPG)")
        
        avg_kobe_assists = calculate_player_kobe_assist_average(player_name, player_id, sample_size)
        
        results[player_name] = {
            'kobe_assists_per_game': avg_kobe_assists,
            'points_per_game': ppg,
            'team': player['TEAM_ABBREVIATION']
        }
        
        print(f"  Final: {avg_kobe_assists:.2f} Kobe assists per game")
        print("-" * 40)
    
    return results


In [70]:
"""Plot Kobe assists vs scoring average"""

def plot_kobe_assists_vs_scoring(results):

    players = list(results.keys())
    kobe_assists = [results[p]['kobe_assists_per_game'] for p in players]
    ppg = [results[p]['points_per_game'] for p in players]
    
    # Bar chart of Kobe assists
    plt.figure(figsize=(12, 8))
    plt.subplot(2, 1, 1)
    plt.bar(players, kobe_assists, color='purple', alpha=0.7)
    plt.ylabel('Kobe Assists per Game')
    plt.title('Top Scorers - Kobe Assists per Game (Play-by-Play Analysis)')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    
    # Scatter plot: PPG vs Kobe Assists
    plt.subplot(2, 1, 2)
    plt.scatter(ppg, kobe_assists, color='red', s=100, alpha=0.7)
    for i, player in enumerate(players):
        plt.annotate(player.split()[-1], (ppg[i], kobe_assists[i]), 
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    plt.xlabel('Points per Game')
    plt.ylabel('Kobe Assists per Game')
    plt.title('Scoring vs Kobe Assists Correlation')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [71]:
"""Print a nice summary table"""

def print_summary_table(results):
    print("\n" + "=" * 70)
    print("FINAL RESULTS - Kobe Assists per Game")
    print("=" * 70)
    print(f"{'Player':<20} {'Team':<5} {'PPG':<6} {'Kobe Assists':<12}")
    print("-" * 70)
    
    # Sort by Kobe assists
    sorted_results = sorted(results.items(), 
                          key=lambda x: x[1]['kobe_assists_per_game'], 
                          reverse=True)
    
    for player, stats in sorted_results:
        print(f"{player:<20} {stats['team']:<5} {stats['points_per_game']:<6.1f} {stats['kobe_assists_per_game']:<12.2f}")


In [72]:
# Main execution
def main():
    # Calculate Kobe assists for top 8 scorers using 10 game samples
    results = calculate_top_scorers_kobe_assists(n_players=8, sample_size=10)
    
    # Display results
    print_summary_table(results)
    
    # Plot results
    plot_kobe_assists_vs_scoring(results)

if __name__ == "__main__":
    main()

NBA Kobe Assist Analysis - Top Scorers
Analyzing top 8 scorers with 10 games each...



KeyError: 'GP'